# Title: Heart Disease .....

### *Introduction*
- ...
- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question

In [7]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



In [8]:
set.seed(3456)
heart_data <- read_csv("data/heart.csv")
heart_data

Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
50,0,0,110,254,0,0,159,0,0.0,2,0,2,1
54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


In [9]:
# select the columns that we want to use
heart_data_selected <- select(heart_data, trestbps, chol, target)
heart_data_selected

trestbps,chol,target
<dbl>,<dbl>,<dbl>
125,212,0
140,203,0
145,174,0
⋮,⋮,⋮
110,275,0
110,254,1
120,188,0


In [22]:
heart_data_scaled <- heart_data_selected |> 
 mutate(scaled_trestbps = scale(trestbps, center = TRUE), 
        scaled_chol = scale(chol, center = TRUE))
heart_data_scaled

trestbps,chol,target,scaled_trestbps,scaled_chol
<dbl>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>"
125,212,0,-0.3774513,-0.6590104
140,203,0,0.4788735,-0.8334543
145,174,0,0.7643151,-1.3955514
⋮,⋮,⋮,⋮,⋮
110,275,0,-1.2337761,0.5620971
110,254,1,-1.2337761,0.1550613
120,188,0,-0.6628929,-1.1241942


#### *Explanation to Why These Columns are Selected*
- ...

#### *What do you expect to find?*

### *Preliminary exploratory data analysis*

In [11]:
#training data
heart_data_split <- initial_split(heart_data, prop = 0.75, strata = target)

<Training/Testing/Total>
<768/257/1025>

In [20]:
# mean of selected predictors
heart_data_mean <- summarize(heart_data_selected, 
                             chol_mean = mean(chol), 
                             trestbps_mean = mean(trestbps))
heart_data_mean

# number of observations for each class
heart_data_observations <- group_by(heart_data_selected, target) |>
    summarize(count = n())
heart_data_observations

# rows with missing data
heart_data_missing <- filter(heart_data_selected, trestbps == NA, chol == NA, target == NA) |>
    count()
heart_data_missing

chol_mean,trestbps_mean
<dbl>,<dbl>
246,131.6117


target,count
<dbl>,<int>
0,499
1,526


n
<int>
0


### *Methods*

### *Expected outcomes and significance*

In [7]:
- What impacts could such findings have?
- What future questions could this lead to?

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: - What impacts
           ^
